In [83]:
%load_ext autoreload
%autoreload 1
import sys; sys.path.append("..")
%aimport lib.util
from lib.util import reformat_tract, add_race_eth, add_purpose_type
import pandas as pd
import numpy as np
from ambry import library


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition

source = p.select("SELECT * FROM lar WHERE county_code = 73").pandas



Figure out what the respondent_id is for Bank of America

In [87]:
ts = l.get('ffiec.gov-hmda-2010e-ts').partition.pandas
ts_name = ts[['respondent_id','respondent_name']]
x = source[['id','respondent_id', 'loan_type']].merge(ts_name, on='respondent_id')
x[x.respondent_name.str.contains('BANK OF AMERICA')][['respondent_id','respondent_name']].drop_duplicates()

,respondent_id,respondent_name
4398,0000012773,BANK OF AMERICAN FORK
17927,0000014740,FIRST NATIONAL BANK OF AMERICA
263517,0000480228,"BANK OF AMERICA, N.A."


In [84]:
sd_msa = lib.util.tract_to_msa()
df = source.set_index('census_tract_number').join(sd_msa, rsuffix='_r').reset_index()
add_race_eth(df)
add_purpose_type(df) # Combines loan_purpose and loan_type in the same way as the summary report


In [85]:

x = df[(df.cbsa == "41740") & ( df.respondent_id == '0000480228' ) ][['purpose_type','action_type','loan_amount']]
x.groupby('purpose_type').count()[['action_type']]

,action_type
purpose_type,
improvement,100
purchase-conventional,948
purchase-govt,516
refinance,7789


In [86]:
x.groupby('purpose_type').sum()[['loan_amount']]

,loan_amount
purpose_type,
improvement,32110
purchase-conventional,449018
purchase-govt,161891
refinance,2584599


In [66]:
df

,census_tract_number,id,year,respondent_id,agency_code,loan_type,property_type,loan_purpose,occupancy,loan_amount,...,application_date_indicator,county,tract,cbsa,cntyname,cbsaname,hus10,afact,AFACT2,race_eth
0,0001.00,1481815,2012,0000011099,5,1,1,3,1,173,...,0,6073,1,41740,San Diego CA,"San Diego-Carlsbad, CA",1397,1,0.001,w
1,0001.00,1481939,2012,0000013303,3,1,1,3,1,300,...,0,6073,1,41740,San Diego CA,"San Diego-Carlsbad, CA",1397,1,0.001,None
2,0001.00,1482118,2012,0000014384,5,1,1,3,1,1123,...,0,6073,1,41740,San Diego CA,"San Diego-Carlsbad, CA",1397,1,0.001,w
3,0001.00,1482973,2012,0000014384,5,1,1,3,2,321,...,0,6073,1,41740,San Diego CA,"San Diego-Carlsbad, CA",1397,1,0.001,w
4,0001.00,1483667,2012,0000014384,5,1,1,3,1,800,...,0,6073,1,41740,San Diego CA,"San Diego-Carlsbad, CA",1397,1,0.001,None
5,0001.00,1483713,2012,0000014384,5,1,1,3,1,175,...,0,6073,1,41740,San Diego CA,"San Diego-Carlsbad, CA",1397,1,0.001,w
6,0001.00,1483879,2012,0000014384,5,1,1,3,1,264,...,0,6073,1,41740,San Diego CA,"San Diego-Carlsbad, CA",1397,1,0.001,None
7,0001.00,1485885,2012,0000015219,3,1,1,1,2,291,...,0,6073,1,41740,San Diego CA,"San Diego-Carlsbad, CA",1397,1,0.001,w
8,0001.00,1486470,2012,0000017464,1,1,1,2,1,180,...,0,6073,1,41740,San Diego CA,"San Diego-Carlsbad, CA",1397,1,0.001,None
9,0001.00,1486919,2012,0000019222,3,1,1,1,1,400,...,0,6073,1,41740,San Diego CA,"San Diego-Carlsbad, CA",1397,1,0.001,w
